In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine




#Dataset manipulation

In [ ]:
#Load Dataset
def getdataset(filepath):
  """Loading the dataset, inspections and cleaning"""
  try:
    df = pd.read_excel(filepath)
    return df
  except FileNotFoundError:
    print(f"Error: File not found at path: {filepath}")
    return None

filepath="Online Retail.xlsx"
df = getdataset(filepath)
#This is a reusable function where filepath differs from the location of datasets

#Inspect
if df is not None:
  print("First few rows of the dataset: \n",df.head())
  print("\n The datatypes in this dataset: \n",df.info())

#Cleaning
#Noticed numerous invalid values in Description, UnitPrice = 0 , -ve values

  print("Initial Dataframe Shape:", df.shape)
  print("Missing Description Count:", df['Description'].isnull().sum())
  print("Zero Unit Price Count:", (df['UnitPrice'] == 0).sum())

#C1. Remove Rows with Missing Descriptions or where Description is whitespace
  df.dropna(subset=['Description'], inplace=True)
  df = df[df['Description'].str.strip() != ""]  # Remove whitespace-only descriptions

#C2. Remove Rows with Zero Unit Prices
  df = df[df['UnitPrice'] != 0]

#C3. Removing rows with invalid values for Quantity
  df = df[pd.to_numeric(df['Quantity'], errors='coerce').notna()]
  df['Quantity'] = pd.to_numeric(df['Quantity'])

  df['IsReturn'] = df['Quantity'] < 0
  df['Quantity'] = df['Quantity'].abs()

#C4. Remove rows where 'StockCode' is not valid.
#Convert 'StockCode' to string type to handle mixed data types
  df['StockCode'] = df['StockCode'].astype(str)
  df = df[df['StockCode'].str.strip() != ""]

# Convert InvoiceDate to datetime (handle different formats)
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
  df.dropna(subset=['InvoiceDate'], inplace=True)

# Inspect (Post-Cleaning)
  print("\nCleaned Dataframe Shape:", df.shape)
  print("Missing Description Count (after cleaning):", df['Description'].isnull().sum())
  print("Zero Unit Price Count (after cleaning):", (df['UnitPrice'] == 0).sum())
  print("Unique Stock Codes:", df['StockCode'].nunique())

# Reset Index (Good Practice After Filtering)
  df.reset_index(drop=True, inplace=True)

else:
  print("Dataset not loaded.")

#More cleaning is necessary as some product description is having jargon value such as
#[amazon, missing,can't find, breakages..etc]

#List Dictionaries

In [ ]:
#Making lists and dictionaries
product_list = df['Description'].unique().tolist()
product_dict = {product_list[i]:i for i in range(len(product_list))}
for key, value in product_dict.items():
    if value > 10:
        print(f"{key}: High Sales ({value})")
    else:
        print(f"{key}: Low Sales ({value})")


#Data Manipulation #Modifications

In [7]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

grouped_data = df.groupby('Description')['TotalPrice'].sum().reset_index()
grouped_data.sort_values(by='TotalPrice', ascending=False, inplace=True)
df.head()

# Filter: Sales in the UK
uk_sales = df[df['Country'] == 'United Kingdom']

# Sort: By UnitPrice in descending order
sorted_by_price = df.sort_values('UnitPrice', ascending=False)

# Group: Sales by Country
sales_by_country = df.groupby('Country')['Quantity'].sum()

#Mathematical Operations

In [8]:
# Calculate total sales value
df['Sales'] = df['Quantity'] * df['UnitPrice']
total_sales = df['Sales'].sum()
print(f"\nTotal Sales Value: {total_sales:.2f}")

#F string does the type-casting automatically to string

# Average unit price
average_price = df['UnitPrice'].mean()
print(f"Average Unit Price: {average_price:.2f}")


Total Sales Value: 11541372.91
Average Unit Price: 4.63


#Data visualization
#Charts and Plots

In [ ]:
# Trends: Sales over time (line chart)
sales_over_time = df.groupby(df['InvoiceDate'].dt.date)['Sales'].sum()
plt.figure(figsize=(10, 6))
plt.plot(sales_over_time.index, sales_over_time.values)
plt.title('Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.grid(True)
plt.show()


In [ ]:
#Box Plots: Visualize the distribution of 'UnitPrice' and 'Quantity'
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['UnitPrice'])
plt.title('Distribution of Unit Price')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x=df['Quantity'])
plt.title('Distribution of Quantity')
plt.show()

In [ ]:
#Scatter Plots: Explore the relationship between 'Quantity' and 'UnitPrice'

plt.figure(figsize=(10, 6))
plt.scatter(df['Quantity'], df['UnitPrice'])
plt.xlabel('Quantity')
plt.ylabel('UnitPrice')
plt.title('Relationship between Quantity and Unit Price')
plt.grid(True)
plt.show()

In [ ]:
#Bar Plot: Visualize the top 10 selling products with TotalPrice
plt.figure(figsize=(10, 5))
sns.barplot(x=grouped_data['Description'][:10], y=grouped_data['TotalPrice'][:10])
plt.xticks(rotation=90)
plt.title("Top 10 Selling Products")
plt.show()


#SQLAlchemy

In [ ]:
# Create SQLite database (you can change to another database)
engine = create_engine('sqlite:///sales_data.db', echo=False)
# In-memory: 'sqlite:///:memory:'

# Store the processed DataFrame in the database
df.to_sql('sales', engine, if_exists='replace', index=False)

# Query: Average sales by product description
query = """
SELECT Description, AVG(Sales) AS AverageSales
FROM sales
GROUP BY Description
ORDER BY AverageSales DESC
LIMIT 10;
"""
with engine.connect() as conn:
    result = pd.read_sql_query(query, conn)

print("\nTop 10 Product Descriptions by Average Sales:\n", result)

#Interpretation

This codespace analyzes an online retail dataset in which the user performs data cleaning, transformations and visual analysis.
Initial steps include importing the dataset, scrutinizing the missing or invalid values present as empty product descriptions, zero unit prices, incorrect quantity and stock codes values.
Data cleansing ensures the accuracy and reliability of business data which is essential to make the data ready for analytics resulting in data-driven business decisions.

In the next phase, user subsets the dataset into a few lists and dictionaries to analyze unique items and their sales volumes.
A key observation is the segmentation of products based on sales frequency (high vs. low sales). This analysis provides valuable insights into product demand, allowing businesses to identify top-selling items and those with lower customer interest